El modelo por excelencia al realizar una recomendacion es la similitud del coseno, de ahi partis una vez tengas vectorizadas tus palabras. Vos podes usar el que quieras, siempre y cuando cumpla la funcion de recomendar.

hagas el 

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
#from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
#from sklearn.neighbors import NearestNeighbors

In [48]:
df_movies=pd.read_csv('../DatosProcesados/movies.csv')

In [49]:
df_movies['combinado'] = df_movies['genres'] +' '+ df_movies['title'] + ' ' + df_movies['overview']


In [50]:
df_movies['combinado'] = df_movies['combinado'].apply( lambda x: re.sub(r'[^\w\s]', '', x) if pd.notnull(x) else '' )


In [51]:
df_movies['combinado'][1]

'Adventure Fantasy Family Jumanji When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world they unwittingly invite Alan  an adult whos been trapped inside the game for 26 years  into their living room Alans only hope for freedom is to finish the game which proves risky as all three find themselves running from giant rhinoceroses evil monkeys and other terrifying creatures'

In [68]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df_movies['combinado'])

In [69]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [70]:
cosine_sim

array([[1.        , 0.00629721, 0.00221233, ..., 0.        , 0.        ,
        0.        ],
       [0.00629721, 1.        , 0.00854614, ..., 0.        , 0.        ,
        0.        ],
       [0.00221233, 0.00854614, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [74]:
item_similarity=pd.DataFrame(cosine_sim,index=df_movies['title'],columns=df_movies['title'])
item_similarity

title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,St. Michael Had a Rooster,House of Horrors,Shadow of the Blair Witch,The Burkittsville 7,Caged Heat 3000,Robin Hood,Century of Birthing,Betrayal,Satan Triumphant,Queerama
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.006297,0.002212,0.000873,0.000927,0.000000,0.001177,0.002648,0.000000,0.000000,...,0.0,0.000000,0.0,0.002685,0.000000,0.004200,0.000000,0.000000,0.0,0.0
Jumanji,0.006297,1.000000,0.008546,0.000000,0.000000,0.014078,0.000000,0.004555,0.012489,0.002594,...,0.0,0.000000,0.0,0.000000,0.001717,0.000000,0.015265,0.000000,0.0,0.0
Grumpier Old Men,0.002212,0.008546,1.000000,0.002459,0.012576,0.000000,0.003316,0.002833,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.003505,0.001625,0.000000,0.000000,0.0,0.0
Waiting to Exhale,0.000873,0.000000,0.002459,1.000000,0.001104,0.002794,0.003693,0.003176,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.003106,0.013353,0.001178,0.001060,0.0,0.0
Father of the Bride Part II,0.000927,0.000000,0.012576,0.001104,1.000000,0.000000,0.009572,0.000000,0.006940,0.000000,...,0.0,0.000000,0.0,0.000000,0.004145,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Robin Hood,0.004200,0.000000,0.001625,0.013353,0.000000,0.002465,0.002441,0.002106,0.002732,0.002186,...,0.0,0.000000,0.0,0.006815,0.001289,1.000000,0.001254,0.004003,0.0,0.0
Century of Birthing,0.000000,0.015265,0.000000,0.001178,0.000000,0.001146,0.000000,0.000980,0.000000,0.011417,...,0.0,0.000000,0.0,0.000000,0.000000,0.001254,1.000000,0.001862,0.0,0.0
Betrayal,0.000000,0.000000,0.000000,0.001060,0.000000,0.012079,0.000000,0.003126,0.019342,0.006237,...,0.0,0.003309,0.0,0.000000,0.010228,0.004003,0.001862,1.000000,0.0,0.0


In [135]:
resul_recomen=item_similarity['Group Sex'].sort_values(ascending=False)[1:6]
resul_recomen.index.tolist()

['Toy Story',
 'Russell Brand - From Addiction to Recovery',
 'La Grande Bouffe',
 'The Neighbor',
 'Five Element Ninjas']

In [ ]:

resul_recomen=item_similarity['Group Sex'].sort_values(ascending=False)[1:6]
resul_recomen.index.tolist()

In [94]:
resul_recomen=item_similarity['Group Sex'].sort_values(ascending=False)[1:6]
data = {'Pelicula': resul_recomen.index, 'Similitud': resul_recomen.values}
df = pd.DataFrame(data)
df

,Pelicula,Similitud
0,Toy Story,0.074584
1,Russell Brand - From Addiction to Recovery,0.072305
2,La Grande Bouffe,0.061076
3,The Neighbor,0.060930
4,Five Element Ninjas,0.051730


In [86]:
def recomendacion(title:str):
    if title not in item_similarity:
        resultados.append("El título no se encuentra en la base de datos.")
    else:
        resul_recomen=item_similarity[title].sort_values(ascending=False)[1:6]
    return resul_recomen

In [ ]:
def recomendacion(title:str):
    if title not in item_similarity:
        resultados.append("El título no se encuentra en la base de datos.")
    else:
        resul_recomen=item_similarity[title].sort_values(ascending=False)[1:6]
    return resul_recomen

In [88]:
type(recomendacion('Toy Story'))

pandas.core.series.Series

In [85]:
def recomendacionl(title:str):
    resultados=[]
    if title not in item_similarity:
        resultados.append("El título no se encuentra en la base de datos.")
    else:
        resul_recomen=item_similarity[title].sort_values(ascending=False)[1:6]
        resultados.append(resul_recomen)
    return resultados

In [82]:
recomendacion('Avatar')

[title
 Avatar 2                                 0.258642
 Avatar: Creating the World of Pandora    0.104477
 Stand by Me Doraemon                     0.100092
 Bloodbrothers                            0.098497
 The War of the Robots                    0.094172
 Name: Avatar, dtype: float64]